In [ ]:
# Import pieces from codeflare-sdk
from codeflare_sdk import Cluster, ClusterConfiguration, TokenAuthentication

In [ ]:
# Create authentication object and log in to desired user account (if not already authenticated)
auth = TokenAuthentication(
    token = "XXXX",
    server = "XXXX",
    skip_tls = False
)
auth.login()


NOTE: We must specify the `image` which will be used in our RayCluster, we recommend you bring your own image which suits your purposes. 
The example here is a community image.

NOTE: By default the SDK uses Kueue as it's scheduling solution to use MCAD set the `mcad=True` option in `ClusterConfiguration`

In [ ]:
# Create our cluster and submit
# The SDK will try to find the name of your default local queue based on the annotation "kueue.x-k8s.io/default-queue": "true"
namespace = "default"
cluster_name = "hfgputest-1"
local_interactive = True

cluster = Cluster(ClusterConfiguration(local_interactive=local_interactive,
                                       namespace=namespace,
                                       name=cluster_name,
                                       num_workers=1,
                                       min_cpus=1,
                                       max_cpus=1,
                                       min_memory=4,
                                       max_memory=4,
                                       num_gpus=0,
                                       image="quay.io/project-codeflare/ray:latest-py39-cu118",
                                       # local_queue="local-queue-name" # Specify the local queue manually
                                    ))

In [ ]:
cluster.up()

In [ ]:
cluster.wait_ready()

### Connect via the rayclient route

In [ ]:
from codeflare_sdk import generate_cert

if local_interactive:
    generate_cert.generate_tls_cert(cluster_name, namespace)
    generate_cert.export_env(cluster_name, namespace)

In [ ]:
import ray

ray.shutdown()
ray.init(address=cluster.local_client_url(), logging_level="DEBUG")

In [ ]:
import math
import ray

@ray.remote
def heavy_calculation_part(num_iterations):
    result = 0.0
    for i in range(num_iterations):
        for j in range(num_iterations):
            for k in range(num_iterations):
                result += math.sin(i) * math.cos(j) * math.tan(k)
    return result
@ray.remote
def heavy_calculation(num_iterations):
    results = ray.get([heavy_calculation_part.remote(num_iterations//30) for _ in range(30)])
    return sum(results)


In [ ]:
ref = heavy_calculation.remote(3000)

In [ ]:
ray.get(ref)

In [ ]:
ray.cancel(ref)
ray.shutdown()

In [ ]:
cluster.down()